In [ ]:
import sys
import os

os.chdir("..")
os.chdir("./src")
# sys.path.append("./src")

In [ ]:
from IPython import display
from IPython.display import clear_output
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, jit
from time import time

# from helpers import *
from LDMIBSS import *

np.random.seed(13)
%load_ext autoreload
%autoreload 2

In [ ]:
N = 500000
# Generate sufficiently scattered source signals
S = np.random.uniform(-0.1, 1.1, size=(5, N))
S = S * (S >= 0) * (S <= 1) + (S > 1) * 1
# Number of Source and Mixtures
NumberofSources = S.shape[0]
NumberofMixtures = 8

# Generate Mxr random mixing from i.i.d N(0,1)
A = np.random.randn(NumberofMixtures, NumberofSources)
X = np.dot(A, S)
X = X + 0.01 * np.random.randn(X.shape[0], X.shape[1])
print("The following is the mixture matrix A")
display_matrix(A)

In [ ]:
subplot_1D_signals(
    S.T[0:100], title="Original Signals", figsize=(15.2, 9), colorcode=None
)
subplot_1D_signals(
    X.T[0:100], title="Mixture Signals", figsize=(15, 18), colorcode=None
)

In [ ]:
s_dim = S.shape[0]
x_dim = X.shape[0]
debug_iteration_point = 1000
model = OnlineLDMIBSS(
    s_dim=s_dim,
    x_dim=x_dim,
    neural_OUTPUT_COMP_TOL=1e-8,
    set_ground_truth=True,
    S=S,
    A=A,
)

In [ ]:
model.fit_batch_nnantisparse(
    X=X,
    n_epochs=1,
    plot_in_jupyter=True,
    neural_lr_start=1.5,
    neural_lr_stop=0.1,
    debug_iteration_point=debug_iteration_point,
    shuffle=True,
)

In [ ]:
mpl.rcParams["xtick.labelsize"] = 18
mpl.rcParams["ytick.labelsize"] = 18
plot_convergence_plot(
    model.SIR_list,
    xlabel="Number of Iterations / {}".format(debug_iteration_point),
    ylabel="SINR (dB)",
    title="SINR Convergence Plot",
    colorcode=None,
    linewidth=1.8,
)

print("Final SIR: {}".format(np.array(model.SIR_list[-1])))

In [ ]:
Wf = model.Wf
Y = Wf @ X
Y.shape, X.shape, S.shape

In [ ]:
Y_ = signed_and_permutation_corrected_sources(S.T, Y.T).T

In [ ]:
subplot_1D_signals(
    X=Y_.T[0:100],
    title="Extracted Signals (Sign and Permutation Corrected)",
    figsize=(15.2, 9),
    colorcode=None,
)

In [ ]:
subplot_1D_signals(
    X=S.T[0:100], title="Original Signals", figsize=(15.2, 9), colorcode=None
)